# Dask Gateway Example Notebook
Here are some commands for starting and working with a Dask Gateway cluster.

## Start Dask cluster

In [1]:
import os
import dask
from dask_gateway import Gateway
from dask.distributed import Client
import dask_gateway
import dask.array as da

In [2]:
# get the dask-gateway version
dask_gateway.__version__

'0.7.1'

In [3]:
# show the default dask-gateway settings
dask.config.config['gateway']

{'auth': {'type': 'jupyterhub', 'kwargs': {}},
 'cluster': {'options': {'image': '{JUPYTER_IMAGE_SPEC}'}},
 'address': 'https://ooi.pangeo.io/services/dask-gateway/',
 'proxy_address': 'gateway://traefik-ooi-prod-dask-gateway.ooi-prod:80',
 'public-address': None,
 'http-client': {'proxy': True}}

In [4]:
# show the current default image to be started on workers
os.environ['JUPYTER_IMAGE_SPEC']

'ooicloud.azurecr.io/ooi-pangeo-io-notebook:1590ac8'

In [5]:
# explicit gateway call
#gateway = Gateway(address = 'https://ooi.pangeo.io/services/dask-gateway/',
#                  proxy_address = 'gateway://traefik-ooi-prod-dask-gateway.ooi-prod:80',
#                  auth = 'jupyterhub')

In [6]:
#default gateway call
gateway = Gateway()

In [7]:
# call new_cluster with explicit image
#cluster = gateway.new_cluster(image=os.environ['JUPYTER_IMAGE'])

In [8]:
# default new_cluster call
cluster = gateway.new_cluster()

In [9]:
gateway.list_clusters()

[ClusterReport<name=ooi-prod.8086d26488bd41daa31d00fb1fe51625, status=RUNNING>]

In [10]:
# get the name of the new cluster
gateway.list_clusters()[0].name

'ooi-prod.8086d26488bd41daa31d00fb1fe51625'

In [11]:
# connect to cluster that already exists
# this is helpful if the cluster exists but it is disconnected for some reason
#cluster_name = gateway.list_clusters()[0].name
#cluster = gateway.connect(cluster_name)

In [12]:
# the dashboard_link property will show the link that can be pasted into the Dask labextension
cluster.dashboard_link

'https://ooi.pangeo.io/services/dask-gateway/clusters/ooi-prod.8086d26488bd41daa31d00fb1fe51625/status'

## Scale cluster and connect a client
"cluster" will show the interactive scaling controls and the dashboard link, which can be pasted into the Dask labextension.

In [13]:
cluster

In [14]:
# scale cluster to 4 workers using the scale() method
cluster.scale(12)

In [15]:
# connect a client
# the distributed client is used for running parallel tasks with Dask
client = Client(cluster)

In [18]:
client

Client Scheduler: gateway://traefik-ooi-prod-dask-gateway.ooi-prod:80/ooi-prod.8086d26488bd41daa31d00fb1fe51625 Dashboard: https://ooi.pangeo.io/services/dask-gateway/clusters/ooi-prod.8086d26488bd41daa31d00fb1fe51625/status,Cluster Workers: 11 Cores: 22 Memory: 47.24 GB


## Test Dask cluster

In [20]:
%%time
a = da.random.normal(size=(10000, 100000), chunks=(500, 500))
f = client.submit(a.mean().compute)
f.result()

CPU times: user 475 ms, sys: 35.4 ms, total: 510 ms
Wall time: 8.26 s


1.1581672621235419e-05

## Other misc commands
These commands are useful for getting information but aren't normally needed

In [21]:
options = gateway.cluster_options()

In [22]:
options

Box(children=(Box(children=(HTML(value='<h2>Cluster Options</h2>'), Box(children=(HTML(value="<p style='font-w…

In [23]:
options.worker_cores = 4

In [24]:
client.scheduler_info()['services']

{'dashboard': 8787, 'gateway': 8788}

In [25]:
#cluster.get_client()

In [26]:
gateway.address

'https://ooi.pangeo.io/services/dask-gateway'

In [27]:
cluster.scheduler_address

'gateway://traefik-ooi-prod-dask-gateway.ooi-prod:80/ooi-prod.8086d26488bd41daa31d00fb1fe51625'

In [28]:
cluster.scheduler_comm

<rpc to 'gateway://traefik-ooi-prod-dask-gateway.ooi-prod:80/ooi-prod.8086d26488bd41daa31d00fb1fe51625', 1 comms>

In [29]:
cluster.scheduler_info

{'type': 'Scheduler',
 'id': 'Scheduler-97871417-c73f-4c51-9260-433f1b5a7999',
 'address': 'tls://10.0.129.77:8786',
 'services': {'dashboard': 8787, 'gateway': 8788},
 'workers': {'tls://10.0.128.205:36335': {'type': 'Worker',
   'id': 'dask-gateway-d7t4p',
   'host': '10.0.128.205',
   'resources': {},
   'local_directory': '/home/jovyan/worker-8g6k81c2',
   'name': 'dask-gateway-d7t4p',
   'nthreads': 2,
   'memory_limit': 4294967296,
   'services': {'dashboard': 45801},
   'nanny': 'tls://10.0.128.205:34923'},
  'tls://10.0.129.176:41475': {'type': 'Worker',
   'id': 'dask-gateway-z9lqm',
   'host': '10.0.129.176',
   'resources': {},
   'local_directory': '/home/jovyan/worker-kdtprtlt',
   'name': 'dask-gateway-z9lqm',
   'nthreads': 2,
   'memory_limit': 4294967296,
   'services': {'dashboard': 41583},
   'nanny': 'tls://10.0.129.176:39343'},
  'tls://10.0.131.150:41273': {'type': 'Worker',
   'id': 'dask-gateway-825gw',
   'host': '10.0.131.150',
   'resources': {},
   'local_di

## Shutdown cluster

In [ ]:
cluster.shutdown()